In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Maximazing the Dsiplay
pd.set_option('display.max_columns', None) 
pd.set_option('display.max_rows', None)
pd.set_option('display.width', None)

from warnings import simplefilter
# ignore all future warnings
simplefilter(action='ignore', category=FutureWarning)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

water_treatment= pd.read_excel('/kaggle/input/water_treatment.xlsx',header=None)
print (water_treatment.head())

# PREPROCESSING

# Droping unneccessary columns
water_treatment= water_treatment.drop([0],axis=1)
print ('Number of columns and rows',water_treatment.shape)
print ('Type of each column',water_treatment.dtypes)


# Missing Values

# replacing '?' by 0
water_treatment=water_treatment.replace('?', 0)
# Converting all in to float
water_treatment = water_treatment.apply(lambda x: x.astype(np.float64), axis=1)
# Now we can have all the relevant statistics of each column
print (water_treatment.describe())

# Replacing '0' of each column with average value
water_treatment=water_treatment.replace(0.0,water_treatment.mean())
print (water_treatment.head())

# For making a target variable, we will take a random value of k'
# K-MEANS CLUSTERING
from sklearn.cluster import KMeans 
clusterNum = 3
k_means = KMeans(init = "k-means++", n_clusters = clusterNum, n_init = 12)
k_means.fit(water_treatment_normalized)
labels = k_means.labels_
print(labels)

water_treatment["Cluster"] = labels
print (water_treatment.head())

# output file
output= water_treatment[["Cluster"]]
print (output.head())

# Autoencoder

import keras
from keras.models import Sequential
from keras.layers import Input, Dense
from keras.models import Model
from keras.optimizers import adam,sgd
from sklearn.model_selection import train_test_split

X=water_treatment.ix[:,0:38].values
Y = water_treatment['Cluster'].values

min_max_scaler = preprocessing.MinMaxScaler()
X = min_max_scaler.fit_transform(X)

# seed for reproducing same results
seed = 20
np.random.seed(seed)

# split the data into training (80%) and testing (20%)
(x_train, x_test, y_train, y_test) = train_test_split(X, Y, test_size=0.20, random_state=seed)

# reduce to 2 features
encoding_dim = 2
input_df = Input(shape=(38,))
encoded = Dense(encoding_dim, activation='relu')(input_df)
decoded = Dense(38, activation='sigmoid')(encoded)

# encoder
autoencoder = Model(input_df, decoded)

# intermediate result
encoder = Model(input_df, encoded)

autoencoder.compile(optimizer='adadelta', loss='MAE',metrics=['accuracy']) # adadelta

history=autoencoder.fit(x_train, x_train,
                epochs=100,
                batch_size=512,
                shuffle=True,
                validation_data=(x_test, x_test))

# Visualizing
import matplotlib.pyplot as plt
# list all data in history
print(history.history.keys())
# summarize history for accuracy
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()
# summarize history for loss
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()













